<a href="https://colab.research.google.com/github/farrukh-saif/Fake-News-Detector/blob/main/Fake_News_Detector.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Imports

In [1]:
import json
import tensorflow as tf
import numpy as np

#Loading Data

In [2]:
with open("fixed_fake_news.json", 'r') as f:
  datastore = json.load(f)

sentences = []
labels = []

for item in datastore:
  sentences.append(item['headline'])
  labels.append(item['is_sarcastic'])

In [3]:
len(sentences), len(labels)

(28619, 28619)

In [4]:
trainingSize = 20000

In [5]:
trainingSentences = sentences[0:trainingSize]
trainingLabels = labels[0:trainingSize]

testingSentences = sentences[trainingSize:]
testingLabels = labels[trainingSize:]

#Tokenising Corpus

In [6]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = 40000, oov_token = '<OOV>')
tokenizer.fit_on_texts(trainingSentences)
len(tokenizer.word_index)

25898

#Converting Sentences To Data

In [7]:
trainingSequences = tokenizer.texts_to_sequences(trainingSentences)
trainingPadded = tf.keras.utils.pad_sequences(trainingSequences, maxlen = 40)

testingSequences = tokenizer.texts_to_sequences(testingSentences)
testingPadded = tf.keras.utils.pad_sequences(testingSequences, maxlen = 40)

In [8]:
trainingPadded = np.array(trainingPadded)
testingPadded = np.array(testingPadded)

trainingLabels = np.array(trainingLabels)
testingLabels = np.array(testingLabels)

#Creating Model

In [11]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(40000, 3, input_length = 40),
    tf.keras.layers.GlobalAveragePooling1D(),
    tf.keras.layers.Dense(15, activation = 'relu'),
    tf.keras.layers.Dense(1, activation = 'sigmoid')
])

model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

#Training Model

In [12]:
model.fit(trainingPadded, trainingLabels, validation_data=(testingPadded, testingLabels), epochs = 10)

Epoch 1/10
625/625 [==============================] - 4s 5ms/step - loss: 0.6614 - accuracy: 0.6109 - val_loss: 0.5634 - val_accuracy: 0.8094
Epoch 2/10
625/625 [==============================] - 3s 5ms/step - loss: 0.4336 - accuracy: 0.8434 - val_loss: 0.3894 - val_accuracy: 0.8296
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 0.3053 - accuracy: 0.8867 - val_loss: 0.3400 - val_accuracy: 0.8568
Epoch 4/10
625/625 [==============================] - 3s 5ms/step - loss: 0.2396 - accuracy: 0.9128 - val_loss: 0.3240 - val_accuracy: 0.8622
Epoch 5/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1921 - accuracy: 0.9320 - val_loss: 0.3234 - val_accuracy: 0.8596
Epoch 6/10
625/625 [==============================] - 3s 5ms/step - loss: 0.1558 - accuracy: 0.9471 - val_loss: 0.3399 - val_accuracy: 0.8530
Epoch 7/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1254 - accuracy: 0.9589 - val_loss: 0.3495 - val_accuracy: 0.8552
Epoch 

#Running Inference

In [22]:
playSentences = ['the planet just crossed another major carbon milestone', "grandmother doesn't care for new priest"]

playSequences = tokenizer.texts_to_sequences(playSentences)
playPadded = tf.keras.utils.pad_sequences(playSequences, maxlen = 40)

[ "Fake" if (x > 0.5) else "Real" for x in model.predict(np.array(playPadded))]

1/1 [==============================] - 0s 22ms/step


['Real', 'Fake']